In [ ]:
# !pip install torchattacks 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
import glob
import random
import torch
import torchvision
import numpy as np
from tqdm.notebook import tqdm
from collections import OrderedDict
from torchvision.models import resnet50
from torchvision import transforms
from torchvision import datasets
from torchattacks import LGV, BIM, MIFGSM, DIFGSM, TIFGSM

In [ ]:
torch.cuda.is_available()

True

In [ ]:
PATH_LGV_MODELS="gdrive/MyDrive/LGV/models/ImageNet/resnet50/cSGD/seed0"
DATA_PATH="gdrive/MyDrive/LGV/data"
BATCH_SIZE_TRAIN=256  # changing batch-size to collect models might require you to tune the LGV learning rate hyperparameter
BATCH_SIZE_TEST=64
N_WORKERS=2 #4
N_EXAMPLES=100 #500

In [ ]:
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Load models

In [ ]:
def add_normalization_layer(model, mean, std):
    """
    Add a data normalization layer to a model
    """
    return torch.nn.Sequential(
        transforms.Normalize(mean=mean, std=std),
        model
    )

In [ ]:
# LGV surrogate
paths_models = glob.glob(f'{PATH_LGV_MODELS}/*.pt')
paths_models = sorted(paths_models)
list_models = []
for path in paths_models:
    model = resnet50()
    model.load_state_dict(torch.load(path)['state_dict'])
    model = add_normalization_layer(model=model, 
                                    mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
    model = model.eval().cuda()
    list_models.append(model)

In [ ]:
len(list_models)

40

In [ ]:
base_model = resnet50()
ckpt = torch.load(f'{PATH_LGV_MODELS}/original/ImageNet-ResNet50-052e7f78e4db--1564492444-1.pth.tar')['state_dict']
new_state_dict = OrderedDict()
for k, v in ckpt.items():
    name = k[7:]  # remove `module.`
    new_state_dict[name] = v

base_model.load_state_dict(new_state_dict)
base_model = add_normalization_layer(model=base_model,
                                     mean=[0.485, 0.456, 0.406], 
                                     std=[0.229, 0.224, 0.225])
base_model = base_model.eval().cuda()

In [ ]:
target_model = resnet50(pretrained=True)
target_model = add_normalization_layer(model=target_model, 
                                       mean=[0.485, 0.456, 0.406], 
                                       std=[0.229, 0.224, 0.225])
target_model = target_model.eval().cuda()

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# Dataloader

In [ ]:
traindir = os.path.join(DATA_PATH, 'train')
transform_train = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor()
        ])
trainset = datasets.ImageFolder(traindir, transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE_TRAIN,
                                          shuffle=True, num_workers=N_WORKERS,
                                          pin_memory=True)
testdir = os.path.join(DATA_PATH, 'validation')
transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])
testset = datasets.ImageFolder(testdir, transform_test)
indices = torch.from_numpy(np.random.choice(len(testset), size=(N_EXAMPLES,),
                                            replace=False))
# select a subset of 500 examples. The original paper selects only original 
# examples that are correctly predicted by the target model, which might explain
# why we obtain slightly different results.

testsubset = torch.utils.data.Subset(testset, indices)
testloader = torch.utils.data.DataLoader(testsubset, batch_size=BATCH_SIZE_TEST,
                                         shuffle=False, num_workers=N_WORKERS,
                                         pin_memory=False)

# Attack

### Target: the pretrained Resnet50 from torchvision

In [ ]:
def report_success_rate(atk):
    """
    Compute the success rate of the provided attack on test images
    """
    correct = 0
    total = 0
    for images, labels in tqdm(testloader, total=len(testloader)):
        images = images
        images, labels = images.cuda(), labels.cuda()
        with torch.no_grad():
            outputs = target_model(images)
            _, predicted = torch.max(outputs.data, 1)
        adv_images = atk(images, predicted)
        with torch.no_grad():
            outputs_adv = target_model(adv_images)
            _, predicted_adv = torch.max(outputs_adv.data, 1)
        total += labels.size(0)
        correct += (predicted_adv == predicted).sum()
    print(f"Success rate of {type(atk).__name__}{'-'+atk.attack_class.__name__ if hasattr(atk, 'attack_class') else ''}: {100 - 100 * float(correct) / total}%\n")

###### LGV + BIM

In [ ]:
atk = LGV(base_model, trainloader, lr=0.05, epochs=10, nb_models_epoch=4, 
          wd=1e-4, attack_class=BIM, eps=4/255, alpha=4/255/10,
          steps=50, verbose=True)
atk.load_models(list_models)  # load our list of collected models

# uncomment the next 2 lines and comment the last one to collect models yourself (10 ImageNet epochs)
#atk.collect_models()
#atk.save_models('models/lgv')

report_success_rate(atk)

  0%|          | 0/2 [00:00<?, ?it/s]

Phase 2: craft adversarial examples with BIM
Success rate of LGV-BIM: 97.0%



In [ ]:
# print("LGV+MI")
# atk = LGV(base_model, trainloader, lr=0.05, epochs=10, nb_models_epoch=4, wd=1e-4, attack_class=MIFGSM, eps=4/255, alpha=4/255/10, steps=50, verbose=False)
# atk.load_models(list_models)
# report_success_rate(atk)

In [ ]:
# print("LGV+DI")
# atk = LGV(base_model, trainloader, lr=0.05, epochs=10, nb_models_epoch=4, wd=1e-4, attack_class=DIFGSM, eps=4/255, alpha=4/255/10, steps=50, verbose=False)
# atk.load_models(list_models)
# report_success_rate(atk)

# Attack Robosutbench

In [ ]:
pip install git+https://github.com/RobustBench/robustbench.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/RobustBench/robustbench.git to /tmp/pip-req-build-sh5bqetz
  Running command git clone --filter=blob:none --quiet https://github.com/RobustBench/robustbench.git /tmp/pip-req-build-sh5bqetz
  Resolved https://github.com/RobustBench/robustbench.git to commit 1a9c24fa69363d8130f8cdf67ca3ce8a7c481aa8
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/fra31/auto-attack.git (to revision c1ec340e54a227c87c6601ada3abe0910ac4a2c0) to /tmp/pip-install-xbbutdi1/autoattack_987a7599af85460eb93560574bd1aaa9
  Running command git clone --filter=blob:none --quiet https://github.com/fra31/auto-attack.git /tmp/pip-install-xbbutdi1/autoattack_987a7599af85460eb93560574bd1aaa9
  Running command git rev-parse -q --verify 'sha^c1ec340e54a227c87c6601ada3abe0910ac4a2c0'
  Running command git fetch -q https://github.com/fra31/auto-attack.git c1ec340e54a227c87c6601ada

In [ ]:
!ls gdrive

MyDrive  Shareddrives


In [ ]:
!cd gdrive/MyDrive/Robustbench\ models

### Target: 
- Standard_R50
- rank: 9 

In [ ]:
from robustbench.utils import load_model

target_model = load_model(model_name='Standard_R50', dataset='imagenet', threat_model='Linf')

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
target_model = add_normalization_layer(model=target_model,
                                     mean=[0.485, 0.456, 0.406], 
                                     std=[0.229, 0.224, 0.225])
target_model = target_model.eval().cuda()

In [ ]:
atk = LGV(base_model, trainloader, lr=0.05, epochs=10, nb_models_epoch=20, 
          wd=1e-4, attack_class=BIM, eps=4/255, alpha=4/255/10,
          steps=50, verbose=True)
atk.load_models(list_models)  # load our list of collected models
report_success_rate(atk)

  0%|          | 0/2 [00:00<?, ?it/s]

Phase 2: craft adversarial examples with BIM
Success rate of LGV-BIM: 7.0%



In [ ]:
atk = LGV(base_model, trainloader, lr=0.05, epochs=10, nb_models_epoch=20, 
          wd=1e-4, attack_class= MIFGSM, eps=4/255, alpha=4/255/10,
          steps=50, verbose=True)
atk.load_models(list_models)  # load our list of collected models
report_success_rate(atk)

  0%|          | 0/2 [00:00<?, ?it/s]

Phase 2: craft adversarial examples with MIFGSM
Success rate of LGV-MIFGSM: 8.0%



In [ ]:
# print("LGV+MI")
# atk = LGV(base_model, trainloader, lr=0.05, epochs=10, nb_models_epoch=4, wd=1e-4, attack_class=MIFGSM, eps=4/255, alpha=4/255/10, steps=50, verbose=False)
# atk.load_models(list_models)
# report_success_rate(atk)

### Target: 
- Salman2020Do_50_2
- rank: 4

In [ ]:
target_model = load_model(model_name='Salman2020Do_50_2', dataset='imagenet', threat_model='Linf')
target_model = add_normalization_layer(model=target_model,
                                     mean=[0.485, 0.456, 0.406], 
                                     std=[0.229, 0.224, 0.225])
target_model = target_model.eval().cuda()

Download started: path=models/imagenet/Linf/Salman2020Do_50_2.pt (gdrive_id=1OT7xaQYljrTr3vGbM37xK9SPoPJvbSKB)
Download finished: path=models/imagenet/Linf/Salman2020Do_50_2.pt (gdrive_id=1OT7xaQYljrTr3vGbM37xK9SPoPJvbSKB)


In [ ]:
atk = LGV(base_model, trainloader, lr=0.05, epochs=10, nb_models_epoch=4, 
          wd=1e-4, attack_class=BIM, eps=4/255, alpha=4/255/10,
          steps=50, verbose=True)
atk.load_models(list_models)  # load our list of collected models
report_success_rate(atk)

  0%|          | 0/2 [00:00<?, ?it/s]

Phase 2: craft adversarial examples with BIM
Success rate of LGV-BIM: 6.0%



### Target: 
- Debenedetti2022Light_XCiT-L12
- rank: 1

In [ ]:
target_model = load_model(model_name='Debenedetti2022Light_XCiT-L12', dataset='imagenet', threat_model='Linf')
target_model = add_normalization_layer(model=target_model,
                                     mean=[0.485, 0.456, 0.406], 
                                     std=[0.229, 0.224, 0.225])
target_model = target_model.eval().cuda()

Downloading: "https://github.com/RobustBench/robustbench/releases/download/v1.1/debenedetti2022light-xcit-l-imagenet-linf.pth.tar" to /root/.cache/torch/hub/checkpoints/debenedetti2022light-xcit-l-imagenet-linf.pth.tar


In [ ]:
atk = LGV(base_model, trainloader, lr=0.05, epochs=10, nb_models_epoch=4, 
          wd=1e-4, attack_class=BIM, eps=4/255, alpha=4/255/10,
          steps=50, verbose=True)
atk.load_models(list_models)  # load our list of collected models
report_success_rate(atk)

  0%|          | 0/2 [00:00<?, ?it/s]

Phase 2: craft adversarial examples with BIM
Success rate of LGV-BIM: 5.0%



### Target: 
- Standard_R50
- rank: 9

In [ ]:
target_model = load_model(model_name='Standard_R50', dataset='imagenet', threat_model='Linf')
target_model = add_normalization_layer(model=target_model,
                                     mean=[0.485, 0.456, 0.406], 
                                     std=[0.229, 0.224, 0.225])
target_model = target_model.eval().cuda()

In [ ]:
atk = LGV(base_model, trainloader, lr=0.05, epochs=10, nb_models_epoch=4, 
          wd=1e-4, attack_class=BIM, eps=4/255, alpha=4/255/10,
          steps=50, verbose=True)
atk.load_models(list_models)  # load our list of collected models
report_success_rate(atk)

  0%|          | 0/2 [00:00<?, ?it/s]

Phase 2: craft adversarial examples with BIM
Success rate of LGV-BIM: 7.0%

